In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from lightgbm import early_stopping

import yaml
from typing import Callable
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

# Описание и импорт данных

Задачей данного проекта является предсказание кассовых сборов фильмов.
Данные были собраны с сайта Кинопоиск с использованием собственного парсера. Были выбраны данные о фильмах США за все годы с первых 1000 страниц сайта (на следующих страницах информации о кассовых сборах практически не было).

Проект был разделен на несколько частей, каждая из которых представляет собой отдельный Jupyter ноутбук:
1. EDA – исследовательский анализ данных, который включает в себя первичный анализ данных, обработку пропусков, преобразование признаков выявление закономерностей и взаимосвязей в данных, формулировка гипотез и визуализация наиболее важных признаков (данный EDA является упрощенной версией исследовательского проекта - ссылка)
2. **Baseline – обучение baseline модели и оценка ее метрик**
3. Tuning – подбор гиперпараметров модели для улучшения качества предсказаний и оценка полученных метрик

**Описания полей:**
- rating - пользовательский рейтинг на Кинопоиске
- production_year - год производства
- director - режиссер
- age_rating - возрастной рейтинг
- duration - продолжительность в минутах
- budget - бюджет фильма
- **target - кассовые сборы фильма**
- target_log - прологарифмированное значение target (распределение более близкое к нормальному)
- main_genre - основной жанр фильма
- month - месяц выпуска фильма
- director_film_count - количество фильмов, которые снял режиссер
- actors_fame - слава актеров, которые снимались в фильме (слава = количество фильмов, в которых снимались актеры)

In [2]:
config_path = '../config/config.yaml'
config = yaml.load(open(config_path, encoding='utf-8'), Loader=yaml.FullLoader)

config_train = config['train']
RAND = config_train['random_state']

In [15]:
df_train = pd.read_csv(config_train['train_data_path'])
df_val = pd.read_csv(config_train['val_data_path'])
df_test = pd.read_csv(config_train['test_data_path'])

In [42]:
df_train.head()

,rating,production_year,director,age_rating,duration,budget,target_log,main_genre,month,director_film_count,actors_fame
0,6.300000,2015,Джон Уоттс,18+,88,8.000000e+05,11.875191,триллер,январь,5,40
1,7.066667,2004,Other,undefined,95,3.113265e+06,10.605173,документальный,сентябрь,1,24
2,7.700000,1998,Энди Теннант,0+,121,2.600000e+07,18.400536,драма,июль,10,46
3,6.700000,2002,Тодд Филлипс,16+,88,2.400000e+07,18.280746,комедия,февраль,11,81
4,6.696041,1999,Other,undefined,100,1.533853e+07,10.203555,драма,апрель,1,41


# Подготовка данных

In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7785 entries, 0 to 7784
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   rating               7785 non-null   float64
 1   production_year      7785 non-null   int64  
 2   director             7785 non-null   object 
 3   age_rating           7785 non-null   object 
 4   duration             7785 non-null   int64  
 5   budget               7785 non-null   float64
 6   target_log           7785 non-null   float64
 7   main_genre           7785 non-null   object 
 8   month                7785 non-null   object 
 9   director_film_count  7785 non-null   int64  
 10  actors_fame          7785 non-null   int64  
dtypes: float64(3), int64(4), object(4)
memory usage: 669.2+ KB


In [17]:
df_train.describe()

,rating,production_year,duration,budget,target_log,director_film_count,actors_fame
count,7785.000000,7785.000000,7785.000000,7.785000e+03,7785.000000,7785.000000,7785.000000
mean,6.482979,2000.653051,102.017213,2.584692e+07,14.742531,5.031985,30.165832
std,0.849815,18.315396,17.948037,3.349345e+07,3.244874,5.849112,25.954593
min,1.400000,1913.000000,0.000000,2.200000e+02,3.401197,1.000000,1.000000
25%,6.000000,1992.000000,91.000000,6.000000e+06,12.302282,1.000000,8.000000
50%,6.600000,2005.000000,99.000000,1.533853e+07,15.273428,3.000000,23.000000
75%,7.066667,2014.000000,110.000000,2.780000e+07,17.295954,7.000000,46.000000
max,8.900000,2024.000000,319.000000,3.785000e+08,21.796118,42.000000,175.000000


In [19]:
df_train.describe(include='object')

,director,age_rating,main_genre,month
count,7785,7785,7785,7785
unique,501,6,25,12
top,Other,18+,драма,сентябрь
freq,4963,2667,2433,1084


- как видим, пропущенных значений в данных нет (они были обработаны на предыдущем этапе) и у всех переменных правильные типы данных
- так как в данном проекте будет использоваться CatBoost и LightGBM, то все признаки типа object нужно перевести в тип category, что избавит нас от бинаризации категориальных признаков

- преобразуем все столбцы типа object в тип category

In [20]:
def transform_to_category(df: pd.DataFrame, cat_cols: list) -> pd.DataFrame:
    '''
    Преобразует столбцы из типа object в тип category
    Parameters
    ------------
    df: pd.DataFrame
        дата фрейм, в котором нужно преобразовать тип стобцов
    cat_cols: list
        список категориальных столбцов
    
    Returns
    -----------
    дата фрейм с преобразованными столбцами
    '''
    df[cat_cols] = df[cat_cols].astype('category')
    return df

In [21]:
df_train = transform_to_category(df_train, config_train['category_cols'])
df_val = transform_to_category(df_val, config_train['category_cols'])
df_test = transform_to_category(df_test, config_train['category_cols'])

- разделим данные на X и y

In [25]:
def split_to_x_y(df: pd.DataFrame, target_col: str) -> tuple:
    '''
    Делит дата фрейм на X и y
    
    Parameters
    -----------
    df: pd.DataFrame
        дата фрейм с данными
    target_col: str
        название таргет переменной
        
    Returns
    ----------
    разделенный дата фрейм на X и y
    '''
    X = df.drop([target_col], axis=1)
    y = df[target_col]
    return X, y

In [29]:
X_train, y_train = split_to_x_y(df_train, config_train['target_column'])
X_val, y_val = split_to_x_y(df_val, config_train['target_column'])
X_test, y_test = split_to_x_y(df_test, config_train['target_column'])

# Baseline модель

В данном проекте в качестве модели было решено использовать **CatBoost** и **LightGBM**, а потом сравнить их результаты на бейзлайне и выбрать лучшую, по таким причинам:
- отлично подходят для анализа табличных данных
- быстро обучаются и имеют высокую точность предсказаний
- умеют обрабатывать категориальные признаки без необходимости их кодирования, что упрощает работу с большим количеством уникальных значений
- хорошо работают с разреженными данными

В качестве метрики для оценки была выбрана **MAE**, так как она более устойчива к выбросам

In [30]:
# создание моделей
cb_baseline = CatBoostRegressor(random_state=RAND,
                                eval_metric="MAE", 
                                objective='MAE',
                                cat_features=config_train['category_cols'])

lgb_baseline = LGBMRegressor(objective='mae', random_state=RAND, verbosity=-1)

In [31]:
# создание набора для валидации
eval_set = [(X_val, y_val)]

# обучение моделей
cb_baseline.fit(X_train,
                y_train,
                eval_set=eval_set,
                verbose=False,
                early_stopping_rounds=100)

lgb_baseline.fit(X_train,
                 y_train,
                 eval_metric="MAE",
                 categorical_feature=config_train['category_cols'],
                 eval_set=eval_set,
                 callbacks=[early_stopping(stopping_rounds=100)]);

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's l1: 1.4146


In [32]:
# получение предсказаний
y_pred_cb = cb_baseline.predict(X_test)
y_pred_lgb = lgb_baseline.predict(X_test)

# Метрики

In [33]:
def r2_adjusted(y_true: np.ndarray, y_pred: np.ndarray,
                X_test: np.ndarray) -> float:
    '''
    Вычисление коэффициента детерминации для множественной регрессии
    
    Parameters
    -----------
    y_test: np.ndarray
        тестовые значения y
    y_pred: np.ndarray
        предсказания модели
    X_test: np.ndarray
        тестовые значения X
    
    Returns
    ----------
    Значение метрики
    '''
    n_objects = len(y_true)
    n_features = X_test.shape[1]
    r2 = r2_score(y_true, y_pred)
    return 1 - (1 - r2) * (n_objects - 1) / (n_objects - n_features - 1)


def wape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    '''
    Вычисление взвешенной абсолютной процентной ошибки
    
    Parameters
    -----------
    y_test: np.ndarray
        тестовые значения y
    y_pred: np.ndarray
        предсказания модели
    
    Returns
    ----------
    Значение метрики
    '''
    return np.sum(np.abs(y_pred - y_true)) * 100 / np.sum(y_true)


def rmsle(y_true: np.ndarray, y_pred: np.ndarray) -> np.float64:
    '''
    Вычисление среднеквадратической логарифмической ошибки
    
    Parameters
    -----------
    y_test: np.ndarray
        тестовые значения y
    y_pred: np.ndarray
        предсказания модели 
        
    Returns
    ----------
    Значение метрики
    '''
    try:
        return np.sqrt(mean_squared_log_error(y_true, y_pred))
    except:
        return None


def get_metrics(y_test: np.ndarray, y_pred: np.ndarray, X_test: np.ndarray,
                name: str) -> pd.DataFrame:
    '''
    Создание таблицы с основными метриками для модели
    
    Parameters
    ----------
    y_test: np.ndarray
        тестовые значения y
    y_pred: np.ndarray
        предсказания модели
    X_test: np.ndarray
        тестовые значения X
    Returns
    ----------
    датафрейм с метриками
    '''
    metrics = pd.DataFrame()

    metrics['model'] = [name]
    metrics['MSE'] = mean_squared_error(y_test, y_pred)
    metrics['RMSE'] = np.sqrt(mean_squared_error(y_test, y_pred))
    metrics['RMSLE'] = rmsle(y_test, y_pred)
    metrics['MAE'] = mean_absolute_error(y_test, y_pred)
    metrics['R2 adjusted'] = r2_adjusted(y_test, y_pred, X_test)
    metrics['WAPE_%'] = wape(y_test, y_pred)

    return metrics

In [34]:
# создание датафрейма с метриками + потенционирование
metrics = pd.concat([
    get_metrics(np.exp(y_train), np.exp(cb_baseline.predict(X_train)), X_train,
                'CatBoostBaselineTrain'),
    get_metrics(np.exp(y_test), np.exp(y_pred_cb), X_test,
                'CatBoostBaselineTest'),
    get_metrics(np.exp(y_train), np.exp(lgb_baseline.predict(X_train)),
                X_train, 'LGBMBaselineTrain'),
    get_metrics(np.exp(y_test), np.exp(y_pred_lgb), X_test, 'LGBMBaselineTest')
])

In [35]:
metrics.set_index('model').style.highlight_min(
    axis=0, color='lightblue').highlight_max(axis=0, color='lightpink')

,MSE,RMSE,RMSLE,MAE,R2 adjusted,WAPE_%
model,,,,,,
CatBoostBaselineTrain,5894928942723778.000000,76778440.611436,1.799804,24208523.565597,0.715322,48.974804
CatBoostBaselineTest,5843035434928676.000000,76439750.358885,1.944247,27809983.773220,0.658584,55.508199
LGBMBaselineTrain,6377899338583602.000000,79861751.411947,1.760601,22592390.786591,0.691999,45.705303
LGBMBaselineTest,5542783863964557.000000,74449874.841833,1.973105,27150351.399408,0.676128,54.191586


- в целом, метрики на трейне и тесте у обоих моделей отличаются, но не очень сильно, то есть переобучение не очень большое
- по значениям MAE, MSE, RMSE и RMSLE сложно что-то сказать, так как их значения напрямую зависят от таргет переменной, но по MAE лучше справилась модель lightgbm, а на остальных метриках - catboost
- по значению R2 можно сказать, что lightgbm немного лучше обьясняет дисперсию таргет переменной на тестовых данных, чем catboost 
- но WAPE (ошибка на регрессии) довольно велика, так что в следующих разделах попытаемся улучшить модель

In [36]:
def check_overfitting(model: CatBoostRegressor | LGBMRegressor, X_train: np.ndarray,
                      y_train: np.ndarray, X_test: np.ndarray,
                      y_test: np.ndarray, metric: Callable,
                      model_name: str) -> None:
    '''
    Проверяет переобучилась ли модель
    
    Parameters
    ----------
    model: CatBoostRegressor | LGBMRegressor
        модель
    X_train: np.ndarray
        тренировочные данные
    y_train: np.ndarray
        тренировочные значения y
    X_test: np.ndarray
        тестовые данные
    y_test: np.ndarray
        тестовые значения y
    metric: Callable
        функция-метрика для оценки переобучения
    model_name: str
        название модели
    
    Returns
    ----------
    Данные о переобучении и метрики на train и test
    '''
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    metric_train = metric(np.exp(y_train), np.exp(y_pred_train))
    metric_test = metric(np.exp(y_test), np.exp(y_pred_test))

    print(f'Рассчет переобучения {model_name}')
    print(f'{metric.__name__} на train: {round(metric_train, 2)}')
    print(f'{metric.__name__} на test: {round(metric_test, 2)}')
    print(
        f'delta = {round((abs(metric_train - metric_test) / metric_test*100), 1)}%'
    )

In [37]:
# проверка на переобучение
check_overfitting(cb_baseline, X_train, y_train, X_test, y_test,
                  mean_absolute_error, 'CatBoostBaseline')

Рассчет переобучения CatBoostBaseline
mean_absolute_error на train: 24208523.57
mean_absolute_error на test: 27809983.77
delta = 13.0%


In [38]:
# проверка на переобучение
check_overfitting(lgb_baseline, X_train, y_train, X_test, y_test,
                  mean_absolute_error, 'LGBMBaseline')

Рассчет переобучения LGBMBaseline
mean_absolute_error на train: 22592390.79
mean_absolute_error на test: 27150351.4
delta = 16.8%


- при использовании MAE в качестве метрики для проверки, catboost показывает переобучение в 13 процентов, а lightgbm в 16.8
- оба результата довольно хорошие, но учитывая значения метрик и то, что они немного лучше у lightgbm, будем использовать его, а переобучение попытаемся уменьшить при подборе параметров

- сохраним данные о метриках

In [43]:
metrics.set_index('model').to_json(config_train['metrics_path'])